# Desafío N° 2 
## 7 - Machine Learning

### Grupo N° 5

    Integrantes:
                Arangue, Marcelo            
                Bardauil, Joaquín              
                Marquez, Hector              
                Neustadt, Alejandro       
                Pero, Felipe   

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

Usaré prop_apartament_dummies.csv, le haré el drop de nulos y crearé la nueva feature que resultan en prop_apartament_regresion. Lo hago así para obtener también las columnas state_name, ciudad y barrio, que servirán para correr las regresiones relativo a dichas zonas

In [2]:
df = pd.read_csv("prop_apartament_dummies.csv")
columnas=['surface_covered_in_m2', 'rooms', 'price_aprox_usd', 'balcon', 'centrico', 'torre', 'Cochera', 'Amenities']
df_drop = df[columnas].dropna()
indices = df_drop.index
df_dpto = df.iloc[indices]
df_dpto['surface_covered_in_m2 * rooms'] = df_dpto['surface_covered_in_m2'] * df_dpto['rooms']

C:\Users\aneustadt\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [3]:
print(df.shape)
print(df_dpto.shape)

(49944, 36)
(38187, 37)


In [4]:
df_dpto[columnas].head()

,surface_covered_in_m2,rooms,price_aprox_usd,balcon,centrico,torre,Cochera,Amenities
0,55.0,1.0,72000.0,0,0,0,0,0
2,40.0,1.0,138000.0,0,0,0,0,1
3,60.0,1.0,195000.0,0,0,0,0,1
4,36.0,1.0,115000.0,1,0,0,0,0
5,30.0,1.0,111700.0,1,0,1,0,1


# Regresiones

El objetivo es automatizar el proceso de obtener modelos dados conjuntos de train y test. El valor final que queremos obtener es un DataFrame en donde cada fila represente a un modelo, junto con su score y su error cuadrático medio

En primer lugar, separamos los conjuntos de train y test y observamos la forma de cada uno

In [5]:
predictoras = ['surface_covered_in_m2', 'rooms', 'surface_covered_in_m2 * rooms', 'balcon', 'centrico', 'torre', 'Cochera', 'Amenities']

X = df_dpto[predictoras]
y = df_dpto.price_aprox_usd

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=123)

conjuntos = [X_train, X_test, y_train, y_test]
indices = [x.index for x in conjuntos]

In [6]:
def formas(X_train, X_test, y_train, y_test):
    print('Test')
    print(X_train.shape, np.array(y_train).ravel().shape)
    print('\n')
    print('Train')
    print(X_test.shape, y_test.shape)
    
formas(X_train, X_test, y_train, y_test)

Test
(30549, 8) (30549,)


Train
(7638, 8) (7638,)


In [7]:
X_test.head()

,surface_covered_in_m2,rooms,surface_covered_in_m2 * rooms,balcon,centrico,torre,Cochera,Amenities
29714,41.0,1.0,41.0,0,0,0,0,1
37420,44.0,1.0,44.0,0,0,0,0,0
32184,158.0,4.0,632.0,1,0,0,1,0
38103,46.0,2.0,92.0,0,1,0,0,1
7713,130.0,4.0,520.0,1,0,0,1,1


Definimos la función ```regresiones```, que genera modelos de regresión lineal, regresión Ridge y regresión Lasso a partir de dos conjuntos X_train e y_train

In [8]:
def regresiones(X_train, y_train):    
    # armamos los parametros para las regresiones
    al_ridge = np.linspace(0.001, 0.5, 500)  
    al_lasso = np.linspace(0.001, 0.5, 500)
    kf = KFold(n_splits=5, shuffle=True)
    
    # instanciamos los objetos correspondientes a cada regresion
    lm = LinearRegression()
    lm_ridge_cv= RidgeCV(alphas=al_ridge, cv=kf, normalize=False)
    lm_lasso_cv = LassoCV(alphas=al_lasso, cv=kf, normalize=False)
    elastic_net = linear_model.ElasticNet(alpha=0.5, normalize=True)
    
    # por ultimo, fiteamos las regresiones a los datos
    lm.fit(X_train, y_train)
    lm_ridge_cv.fit(X_train, y_train)
    lm_lasso_cv.fit(X_train, y_train)
    elastic_net.fit(X_train, y_train)
    
    return lm, lm_ridge_cv, lm_lasso_cv, elastic_net

Luego, definimos la función ```extraer_scores``` que, dado un modelo, devuelva su nombre, y el R2 y RMSE para train y test

In [9]:
def extraer_scores(modelo, X_train, X_test, y_train, y_test, variables):
    nombre = type(modelo)
    
    rmse = lambda y, y_pred: np.sqrt(mean_squared_error(y, y_pred))
    
    #datos para train
    R2_train = modelo.score(X_train, y_train)

    y_pred_tr_modelo = modelo.predict(X_train)
    RMSE_train = rmse(y_train,y_pred_tr_modelo)
    
    #datos para test
    R2_test = modelo.score(X_test, y_test)

    y_pred_modelo = modelo.predict(X_test)
    RMSE_test = rmse(y_test, y_pred_modelo)
    
    #el ouput será una Serie de pandas. Para facilitar la legibilidad, los juntamos primero en una lista
    datos = [nombre, R2_train, RMSE_train , R2_test, RMSE_test]
    return pd.Series(datos, index=variables)

#variables será una lista con los nombres de los datos que queremos saber
#en particular, está pensado para que sea ['Modelo', 'R2 train', 'RMSE train', 'R2 test', 'RMSE test']
#aparece en la prueba

La función ```df_scores``` devolverá el dataframe deseado, para el que cada fila representará un modelo particular con los datos que queremos saber

In [10]:
# el parámetro modelos será una lista de modelos
def df_scores(modelos, X_train, X_test, y_train, y_test, variables):
    dataframe = pd.DataFrame(columns=variables)
    
    for model in modelos:
        datos = extraer_scores(model, X_train, X_test, y_train, y_test, variables)
        dataframe = dataframe.append(datos, ignore_index=True)
        
    return dataframe

### Prueba 
Probamos la función ```df_scores``` para el DataFrame df_depto

In [11]:
#primero creamos los modelos llamando a la función regresiones
#pasamos como parámetros los conjuntos X_train e y_train que creamos más arriba
lm, lm_ridge_cv, lm_lasso_cv, elastic_net = regresiones(X_train, y_train)

#dado que el parámetro modelos de df_scores debe ser una lista, creamos una lista con los modelos
modelos = [lm, lm_ridge_cv, lm_lasso_cv, elastic_net]

#definimos también las variables que van a funcionar como columnas del dataframe
variables = ['Modelo', 'R2 train', 'RMSE train', 'R2 test', 'RMSE test']

#luego obtenemos el dataframe 
df_scores(modelos,X_train, X_test, y_train, y_test, variables)

,Modelo,R2 train,RMSE train,R2 test,RMSE test
0,<class 'sklearn.linear_model.base.LinearRegres...,0.739475,77644.644973,0.688049,76687.118785
1,<class 'sklearn.linear_model.ridge.RidgeCV'>,0.739475,77644.645027,0.688050,76687.003690
2,<class 'sklearn.linear_model.coordinate_descen...,0.739475,77644.645028,0.688050,76687.024611
3,<class 'sklearn.linear_model.coordinate_descen...,0.000450,152085.932532,-0.000803,137357.827291


# Regresiones por barrio

Ahora falta armar una función que tome los DataFrames en los barrios que tengan más de una cierta cantidad de registros. 
Antes de crear esta funcón, veremos cuántos registros hay por barrio, ciudad y provincia

La función ```print_barrios_a_calcular``` devuelve directamente la cantidad de registros por barrio, ciudad y provincia

In [12]:
def print_barrios_a_calcular(df):
    barrios = df.barrio.unique()
    ciudades = df.ciudad.unique()
    provincias = df.state_name.unique()
    
    print('Tamaño total del dataset: %d' % len(df))
    
    print('\n')
    
    print('Barrios con más de 750 registros')
    registros_barrio = 0 #acá se acumulará el total de registros por barrio
    for barrio in range(len(barrios)):
        mask_barrio = df.barrio == barrios[barrio]
        if len(df[mask_barrio]) > 750:
            print(barrios[barrio], len(df[mask_barrio]))
            registros_barrio += len(df[mask_barrio])
    print('Total: %d' % registros_barrio)
    print('Diferencia: %d' % (len(df) - registros_barrio))
    
    print('\n')
    
    print('Ciudades con más de 750 registros')
    registros_ciudad = 0 #total de registros por ciudad
    for ciudad in range(len(ciudades)):
        mask_ciudad = df.ciudad == ciudades[ciudad]
        if len(df[mask_ciudad]) > 750:
            print(ciudades[ciudad], len(df[mask_ciudad]))
            registros_ciudad += len(df[mask_ciudad])
    print('Total: %d' % registros_ciudad)
    print('Diferencia: %d' % (len(df) - registros_ciudad))
    
    print('\n')
    
    print('Provincias con más de 750 registros')
    registros_provincia = 0 #total de registros por provincia
    for provincia in range(len(provincias)):
        mask_provincia = df.state_name == provincias[provincia]
        if len(df[mask_provincia]) > 750:
            print(provincias[provincia], len(df[mask_provincia]))
            registros_provincia += len(df[mask_provincia])
    print('Total: %d' % registros_provincia)
    print('Diferencia: %d' % (len(df) - registros_provincia))

In [13]:
print_barrios_a_calcular(df_dpto)

Tamaño total del dataset: 38187


Barrios con más de 750 registros
Belgrano 1733
Nordelta 1622
Palermo 1661
Flores 834
Caballito 1594
Mar del Plata 3426
Barrio Norte 753
Recoleta 1039
Villa Crespo 876
Rosario 2045
Villa Urquiza 1007
San Telmo 770
Tigre 986
Total: 18346
Diferencia: 19841


Ciudades con más de 750 registros
Belgrano 1733
Palermo 2316
Tigre 2779
Morón 795
Flores 834
Caballito 1594
Mar del Plata 4009
Barrio Norte 753
La Matanza 812
Recoleta 1039
Villa Crespo 876
Rosario 2045
Villa Urquiza 1007
Vicente López 1430
San Telmo 770
Lomas de Zamora 810
Total: 23602
Diferencia: 14585


Provincias con más de 750 registros
Capital Federal 18295
Bs.As. G.B.A. Zona Norte 6867
Bs.As. G.B.A. Zona Oeste 2301
Buenos Aires Costa Atlántica 4740
Santa Fe 2163
Bs.As. G.B.A. Zona Sur 2548
Total: 36914
Diferencia: 1273


Hubo poca diferencia entre los registros en provincias que tuviesen más de 750 registros. Veamos en qué provincias no se cumple la condición

In [14]:
provincias = df_dpto.state_name.unique()
indices_registros = []

for provincia in provincias:
    mask_provincia = df_dpto.state_name == provincia
    if len(df_dpto[mask_provincia]) < 750: #como se ve, la condición es la inversa a la que se pedía antes.
        print(provincia, len(df_dpto[mask_provincia]))

Córdoba 626
Neuquén 64
Buenos Aires Interior 273
Río Negro 106
Chubut 7
Salta 15
Entre Ríos 15
Tierra Del Fuego 4
Tucumán 7
Misiones 48
Mendoza 29
Corrientes 69
San Luis 5
Chaco 4
La Pampa 1


## Condicionales
Para los registros que no cumplen esa condición, se buscará la ciudad en la que están, y si la ciudad cumple la condición de la cantidad de registros. Por último, se hará el mismo proceso para provincia.<br>
En los casos en que la condición se cumpla, se usará el DataFrame resultante para entrenar un modelo. Para eso, se lo separará en train_test_split, y se llamará a la función ```regresiones ```

In [15]:
def train_por_zona(df, predictoras):
    barrios = df.barrio.unique()
    ciudades = df.ciudad.unique()
    provincias = df.state_name.unique()
    dataframe = pd.DataFrame()
    
    for barrio in barrios:
        mask_barrio = df.barrio == barrio
        
        #para cada barrio del df, evaluamos la condición de que tenga más de 750 registros
        if len(df[mask_barrio]) > 750:
            
            #si cumple la función, entonces separamos conjuntos de train y test sobre este df
            X = df.loc[mask_barrio, predictoras]
            y = df[mask_barrio].price_aprox_usd
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=123)
            
            #luego, armamos modelos con los conjuntos de train
            lm, lm_ridge_cv, lm_lasso_cv, elastic_net = regresiones(X_train, y_train)
            modelos = [lm, lm_ridge_cv, lm_lasso_cv, elastic_net]
            
            #creamos el índice complejo para el barrio
            iterables = [[barrio], ['Lineal', 'Ridge', 'Lasso', 'ElasticNet']]
            multi_indice = pd.MultiIndex.from_product(iterables, names=['Zona', 'Regresión'])
            
            #el df resultante tendrá una fila por modelo para este barrio
            dataframe_barrio = df_scores(modelos, X_train, X_test, y_train, y_test, variables)
            dataframe_barrio.index = multi_indice
            #luego, sumamos el df de este barrio al df común
            dataframe = dataframe.append(dataframe_barrio, ignore_index=False)
    
    
    #los mismos comentarios aplican al caso de ciudades y provincias
    
    for ciudad in ciudades:
        mask_ciudad = df.ciudad == ciudad
        
        if len(df[mask_ciudad]) > 750:
            X = df.loc[mask_ciudad, predictoras]
            y = df[mask_ciudad].price_aprox_usd
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=123)
            
            lm, lm_ridge_cv, lm_lasso_cv, elastic_net = regresiones(X_train, y_train)
            modelos = [lm, lm_ridge_cv, lm_lasso_cv, elastic_net]
            
            iterables = [[ciudad], ['Lineal', 'Ridge', 'Lasso', 'ElasticNet']]
            multi_indice = pd.MultiIndex.from_product(iterables, names=['Zona', 'Regresión'])
            
            dataframe_ciudad = df_scores(modelos, X_train, X_test, y_train, y_test, variables)
            dataframe_ciudad.index = multi_indice
            dataframe = dataframe.append(dataframe_ciudad, ignore_index=False)
    
    
    for provincia in provincias:
        mask_provincia = df.state_name == provincia
        
        if len(df[mask_provincia]) > 750:
            X = df.loc[mask_provincia, predictoras]
            y = df[mask_provincia].price_aprox_usd
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=123)
            
            lm, lm_ridge_cv, lm_lasso_cv, elastic_net = regresiones(X_train, y_train)
            modelos = [lm, lm_ridge_cv, lm_lasso_cv, elastic_net]
            
            iterables = [[provincia], ['Lineal', 'Ridge', 'Lasso', 'ElasticNet']]
            multi_indice = pd.MultiIndex.from_product(iterables, names=['Zona', 'Regresión'])
            
            dataframe_provincia = df_scores(modelos, X_train, X_test, y_train, y_test, variables)
            dataframe_provincia.index = multi_indice
            dataframe = dataframe.append(dataframe_provincia, ignore_index=False)
            
    dataframe.drop('Modelo', axis=1)
    return dataframe

In [16]:
df_final = train_por_zona(df_dpto, predictoras)

C:\Users\aneustadt\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\aneustadt\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\aneustadt\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\a

In [17]:
df_final.drop('Modelo', axis=1)

R2 train     RMSE train   R2 test  \
Zona                         Regresión                                       
Belgrano                     Lineal      0.819940  101940.518284  0.847340   
                             Ridge       0.819940  101940.541007  0.847302   
                             Lasso       0.819940  101940.518334  0.847339   
                             ElasticNet  0.012569  238721.725014  0.012051   
Nordelta                     Lineal      0.597140   64239.879652  0.576370   
                             Ridge       0.597140   64239.891616  0.576306   
                             Lasso       0.597140   64239.879890  0.576361   
                             ElasticNet  0.008347  100787.746298  0.008232   
Palermo                      Lineal      0.841435   89537.194778  0.817956   
                             Ridge       0.841435   89537.246281  0.817900   
                             Lasso       0.841435   89537.194837  0.817955   
                             ElasticNet  0.011920  223509.623421  0.007099   
Flores                       Lineal      0.806269   44142.230573  0.824500   
                             Ridge       0.806268   44142.271771  0.824546   
                             Lasso       0.806269   44142.230770  0.824506   
                             ElasticNet  0.023314   99113.286063  0.020854   
Caballito                    Lineal      0.820561   44289.052008  0.809591   
                             Ridge       0.820560   44289.065353  0.809588   
                             Lasso       0.820561   44289.052131  0.809590   
                             ElasticNet  0.012772  103883.406143 -0.023225   
Mar del Plata                Lineal      0.732285   39854.910721  0.676379   
                             Ridge       0.732285   39854.915315  0.676375   
                             Lasso       0.732285   39854.910898  0.676380   
                             ElasticNet  0.005764   76805.089028  0.005626   
Barrio Norte                 Lineal      0.835701   97554.120126  0.790042   
                             Ridge       0.835695   97555.747147  0.790539   
                             Lasso       0.835701   97554.120194  0.790044   
                             ElasticNet  0.027129  237386.291607  0.016219   
Recoleta                     Lineal      0.853117  120806.937573  0.810537   
                             Ridge       0.853114  120808.026313  0.810569   
...                                           ...            ...       ...   
San Telmo                    Lasso       0.648835   43618.807027  0.608010   
                             ElasticNet  0.018011   72941.008244  0.019979   
Lomas de Zamora              Lineal      0.804343   28348.829985  0.752999   
                             Ridge       0.804342   28348.881641  0.752896   
                             Lasso       0.804343   28348.830204  0.752996   
                             ElasticNet  0.023005   63348.094959  0.013578   
Capital Federal              Lineal      0.776309   87753.407823  0.767164   
                             Ridge       0.776309   87753.408162  0.767166   
                             Lasso       0.776309   87753.407878  0.767165   
                             ElasticNet  0.001006  185447.455014  0.000703   
Bs.As. G.B.A. Zona Norte     Lineal      0.638119   73580.221238  0.574443   
                             Ridge       0.638119   73580.221637  0.574450   
                             Lasso       0.638119   73580.221238  0.574443   
                             ElasticNet  0.002116  122185.159096 -0.001538   
Bs.As. G.B.A. Zona Oeste     Lineal      0.769479   33888.130366  0.618070   
                             Ridge       0.769479   33888.144597  0.618141   
                             Lasso       0.769479   33888.130576  0.618076   
                             ElasticNet  0.005920   70372.595499  0.008123   
Buenos Aires Costa Atlántica Lineal      0.719406   41568.12692

## Algunas observaciones sobre df_final 

### Máximos y mínimos 

En cuanto a los maximos valores estuvieron parejas

In [18]:
print('Train')
print(df_final['R2 train'].max(level='Regresión'))
print('\n')
print('Test')
print(df_final['R2 test'].max(level='Regresión'))

Train
Regresión
Lineal        0.853117
Ridge         0.853114
Lasso         0.853117
ElasticNet    0.027129
Name: R2 train, dtype: float64


Test
Regresión
Lineal        0.881992
Ridge         0.881987
Lasso         0.881992
ElasticNet    0.023693
Name: R2 test, dtype: float64


Hubo más diferencia en las peores. Pero de todos modos, los resultados no son muy malos

In [19]:
print('Train')
print(df_final['R2 train'].min(level='Regresión'))
print('\n')
print('Test')
print(df_final['R2 test'].min(level='Regresión'))

Train
Regresión
Lineal        0.597140
Ridge         0.597140
Lasso         0.597140
ElasticNet    0.001006
Name: R2 train, dtype: float64


Test
Regresión
Lineal        0.462661
Ridge         0.462661
Lasso         0.462661
ElasticNet   -0.023225
Name: R2 test, dtype: float64


In [20]:
print(max(df_final['R2 train']))
print(max(df_final['R2 test']))
print(min(df_final['R2 train']))
print(min(df_final['R2 test']))

0.8531170335979037
0.8819920081731022
0.001006181429437536
-0.023224893653741363


### Media de los scores

Promedio de los scores para train y test

In [21]:
print(df_final['R2 train'].values.mean())
print(df_final['R2 test'].values.mean())

0.5770268470401673
0.5502288013213233


### DFs por barrio, ciudad y provincia 

In [22]:
barrios = [x for x in df_dpto.barrio.unique() if len(df_dpto[df_dpto.barrio == x]) > 750]
ciudad = [x for x in df_dpto.ciudad.unique() if len(df_dpto[df_dpto.ciudad == x]) > 750]
provincias = [x for x in df_dpto.state_name.unique() if len(df_dpto[df_dpto.state_name == x]) > 750]

In [23]:
df_final.loc[barrios]

Modelo  \
Zona          Regresión                                                       
Belgrano      Lineal      <class 'sklearn.linear_model.base.LinearRegres...   
              Ridge            <class 'sklearn.linear_model.ridge.RidgeCV'>   
              Lasso       <class 'sklearn.linear_model.coordinate_descen...   
              ElasticNet  <class 'sklearn.linear_model.coordinate_descen...   
Nordelta      Lineal      <class 'sklearn.linear_model.base.LinearRegres...   
              Ridge            <class 'sklearn.linear_model.ridge.RidgeCV'>   
              Lasso       <class 'sklearn.linear_model.coordinate_descen...   
              ElasticNet  <class 'sklearn.linear_model.coordinate_descen...   
Palermo       Lineal      <class 'sklearn.linear_model.base.LinearRegres...   
              Ridge            <class 'sklearn.linear_model.ridge.RidgeCV'>   
              Lasso       <class 'sklearn.linear_model.coordinate_descen...   
              ElasticNet  <class 'sklearn.linear_model.coordinate_descen...   
Flores        Lineal      <class 'sklearn.linear_model.base.LinearRegres...   
              Ridge            <class 'sklearn.linear_model.ridge.RidgeCV'>   
              Lasso       <class 'sklearn.linear_model.coordinate_descen...   
              ElasticNet  <class 'sklearn.linear_model.coordinate_descen...   
Caballito     Lineal      <class 'sklearn.linear_model.base.LinearRegres...   
              Ridge            <class 'sklearn.linear_model.ridge.RidgeCV'>   
              Lasso       <class 'sklearn.linear_model.coordinate_descen...   
              ElasticNet  <class 'sklearn.linear_model.coordinate_descen...   
Mar del Plata Lineal      <class 'sklearn.linear_model.base.LinearRegres...   
              Ridge            <class 'sklearn.linear_model.ridge.RidgeCV'>   
              Lasso       <class 'sklearn.linear_model.coordinate_descen...   
              ElasticNet  <class 'sklearn.linear_model.coordinate_descen...   
Barrio Norte  Lineal      <class 'sklearn.linear_model.base.LinearRegres...   
              Ridge            <class 'sklearn.linear_model.ridge.RidgeCV'>   
              Lasso       <class 'sklearn.linear_model.coordinate_descen...   
              ElasticNet  <class 'sklearn.linear_model.coordinate_descen...   
Recoleta      Lineal      <class 'sklearn.linear_model.base.LinearRegres...   
              Ridge            <class 'sklearn.linear_model.ridge.RidgeCV'>   
...                                                                     ...   
Caballito     Lasso       <class 'sklearn.linear_model.coordinate_descen...   
              ElasticNet  <class 'sklearn.linear_model.coordinate_descen...   
Mar del Plata Lineal      <class 'sklearn.linear_model.base.LinearRegres...   
              Ridge            <class 'sklearn.linear_model.ridge.RidgeCV'>   
              Lasso       <class 'sklearn.linear_model.coordinate_descen...   
              ElasticNet  <class 'sklearn.linear_model.coordinate_descen...   
Barrio Norte  Lineal      <class 'sklearn.linear_model.base.LinearRegres...   
              Ridge            <class 'sklearn.linear_model.ridge.RidgeCV'>   
              Lasso       <class 'sklearn.linear_model.coordinate_descen...   
              ElasticNet  <class 'sklearn.linear_model.coordinate_descen...   
Recoleta      Lineal      <class 'sklearn.linear_model.base.LinearRegres...   
              Ridge            <class 'sklearn.linear_model.ridge.RidgeCV'>   
              Lasso       <class 'sklearn.linear_model.coordinate_descen...   
              ElasticNet  <class 'sklearn.linear_model.coordinate_descen...   
Villa Crespo  Lineal      <class 'sklearn.linear_model.base.LinearRegres...   
              Ridge            <class 'sklearn.linear_model.ridge.RidgeCV'>   
              Lasso       <class 'sklearn.linear_model.coordinate_descen...   
              ElasticNet  <class 'sklearn.linear_model.coordinate_descen...   
Rosario       Lineal      <class 'sklear

In [24]:
df_final.loc[ciudad]

Modelo  \
Zona            Regresión                                                       
Belgrano        Lineal      <class 'sklearn.linear_model.base.LinearRegres...   
                Ridge            <class 'sklearn.linear_model.ridge.RidgeCV'>   
                Lasso       <class 'sklearn.linear_model.coordinate_descen...   
                ElasticNet  <class 'sklearn.linear_model.coordinate_descen...   
Palermo         Lineal      <class 'sklearn.linear_model.base.LinearRegres...   
                Ridge            <class 'sklearn.linear_model.ridge.RidgeCV'>   
                Lasso       <class 'sklearn.linear_model.coordinate_descen...   
                ElasticNet  <class 'sklearn.linear_model.coordinate_descen...   
Flores          Lineal      <class 'sklearn.linear_model.base.LinearRegres...   
                Ridge            <class 'sklearn.linear_model.ridge.RidgeCV'>   
                Lasso       <class 'sklearn.linear_model.coordinate_descen...   
                ElasticNet  <class 'sklearn.linear_model.coordinate_descen...   
Caballito       Lineal      <class 'sklearn.linear_model.base.LinearRegres...   
                Ridge            <class 'sklearn.linear_model.ridge.RidgeCV'>   
                Lasso       <class 'sklearn.linear_model.coordinate_descen...   
                ElasticNet  <class 'sklearn.linear_model.coordinate_descen...   
Mar del Plata   Lineal      <class 'sklearn.linear_model.base.LinearRegres...   
                Ridge            <class 'sklearn.linear_model.ridge.RidgeCV'>   
                Lasso       <class 'sklearn.linear_model.coordinate_descen...   
                ElasticNet  <class 'sklearn.linear_model.coordinate_descen...   
Barrio Norte    Lineal      <class 'sklearn.linear_model.base.LinearRegres...   
                Ridge            <class 'sklearn.linear_model.ridge.RidgeCV'>   
                Lasso       <class 'sklearn.linear_model.coordinate_descen...   
                ElasticNet  <class 'sklearn.linear_model.coordinate_descen...   
Recoleta        Lineal      <class 'sklearn.linear_model.base.LinearRegres...   
                Ridge            <class 'sklearn.linear_model.ridge.RidgeCV'>   
                Lasso       <class 'sklearn.linear_model.coordinate_descen...   
                ElasticNet  <class 'sklearn.linear_model.coordinate_descen...   
Villa Crespo    Lineal      <class 'sklearn.linear_model.base.LinearRegres...   
                Ridge            <class 'sklearn.linear_model.ridge.RidgeCV'>   
...                                                                       ...   
La Matanza      Lasso       <class 'sklearn.linear_model.coordinate_descen...   
                ElasticNet  <class 'sklearn.linear_model.coordinate_descen...   
Recoleta        Lineal      <class 'sklearn.linear_model.base.LinearRegres...   
                Ridge            <class 'sklearn.linear_model.ridge.RidgeCV'>   
                Lasso       <class 'sklearn.linear_model.coordinate_descen...   
                ElasticNet  <class 'sklearn.linear_model.coordinate_descen...   
Villa Crespo    Lineal      <class 'sklearn.linear_model.base.LinearRegres...   
                Ridge            <class 'sklearn.linear_model.ridge.RidgeCV'>   
                Lasso       <class 'sklearn.linear_model.coordinate_descen...   
                ElasticNet  <class 'sklearn.linear_model.coordinate_descen...   
Rosario         Lineal      <class 'sklearn.linear_model.base.LinearRegres...   
                Ridge            <class 'sklearn.linear_model.ridge.RidgeCV'>   
                Lasso       <class 'sklearn.linear_model.coordinate_descen...   
                ElasticNet  <class 'sklearn.linear_model.coordinate_descen...   
Villa Urquiza   Lineal      <class 'sklearn.linear_model.base.LinearRegres...   
                Ridge            <class 'sklearn.linear_model.ridge.RidgeCV'>   
                Lasso       <class 'sklearn.linear_model.coordinate_descen...   
                Elast

In [25]:
df_final.loc[provincias]

Modelo  \
Zona                         Regresión                                                       
Capital Federal              Lineal      <class 'sklearn.linear_model.base.LinearRegres...   
                             Ridge            <class 'sklearn.linear_model.ridge.RidgeCV'>   
                             Lasso       <class 'sklearn.linear_model.coordinate_descen...   
                             ElasticNet  <class 'sklearn.linear_model.coordinate_descen...   
Bs.As. G.B.A. Zona Norte     Lineal      <class 'sklearn.linear_model.base.LinearRegres...   
                             Ridge            <class 'sklearn.linear_model.ridge.RidgeCV'>   
                             Lasso       <class 'sklearn.linear_model.coordinate_descen...   
                             ElasticNet  <class 'sklearn.linear_model.coordinate_descen...   
Bs.As. G.B.A. Zona Oeste     Lineal      <class 'sklearn.linear_model.base.LinearRegres...   
                             Ridge            <class 'sklearn.linear_model.ridge.RidgeCV'>   
                             Lasso       <class 'sklearn.linear_model.coordinate_descen...   
                             ElasticNet  <class 'sklearn.linear_model.coordinate_descen...   
Buenos Aires Costa Atlántica Lineal      <class 'sklearn.linear_model.base.LinearRegres...   
                             Ridge            <class 'sklearn.linear_model.ridge.RidgeCV'>   
                             Lasso       <class 'sklearn.linear_model.coordinate_descen...   
                             ElasticNet  <class 'sklearn.linear_model.coordinate_descen...   
Santa Fe                     Lineal      <class 'sklearn.linear_model.base.LinearRegres...   
                             Ridge            <class 'sklearn.linear_model.ridge.RidgeCV'>   
                             Lasso       <class 'sklearn.linear_model.coordinate_descen...   
                             ElasticNet  <class 'sklearn.linear_model.coordinate_descen...   
Bs.As. G.B.A. Zona Sur       Lineal      <class 'sklearn.linear_model.base.LinearRegres...   
                             Ridge            <class 'sklearn.linear_model.ridge.RidgeCV'>   
                             Lasso       <class 'sklearn.linear_model.coordinate_descen...   
                             ElasticNet  <class 'sklearn.linear_model.coordinate_descen...   

                                         R2 train     RMSE train   R2 test  \
Zona                         Regresión                                       
Capital Federal              Lineal      0.776309   87753.407823  0.767164   
                             Ridge       0.776309   87753.408162  0.767166   
                             Lasso       0.776309   87753.407878  0.767165   
                             ElasticNet  0.001006  185447.455014  0.000703   
Bs.As. G.B.A. Zona Norte     Lineal      0.638119   73580.221238  0.574443   
                             Ridge       0.638119   73580.221637  0.574450   
                             Lasso       0.638119   73580.221238  0.574443   
                             ElasticNet  0.002116  122185.159096 -0.001538   
Bs.As. G.B.A. Zona Oeste     Lineal      0.769479   33888.130366  0.618070   
                             Ridge       0.769479   33888.144597  0.618141   
                             Lasso       0.769479   33888.130576  0.618076   
                             ElasticNet  0.005920   70372.595499  0.008123   
Buenos Aires Costa Atlántica Lineal      0.719406   41568.126926  0.717125   
                             Ridge       0.719406   41568.130656  0.717141   
                             Lasso       0.719406   41568.127071  0.717128   
                             ElasticNet  0.003928   78318.841611  0.004116   
Santa Fe                     Lineal      0.767135   38218.069006  0.741115   
                             Ridge       0.767134   38218.084085  0.741185   
                             Lasso       0.767135   38218.069006  0.741115   
 

In [26]:
set(df_final.loc[ciudad].index.levels[0]) - set(df_final.loc[barrios].index.levels[0])

set()

Provincia, ciudad y barrios tienen la misma cantidad de índices. Es decir, hay repetidos. Pero hay diferencias

In [27]:
print(len(set(df_final.loc[ciudad].index.levels[0])))
print(len(set(df_final.loc[barrios].index.levels[0])))
print(len(set(df_final.loc[provincias].index.levels[0])))

23
23
23
